In [5]:
import pandas as pd
import os
from collections import deque
#pd.set_option('display.expand_frame_repr', False)
#pd.options.display.max_rows = 10

def read_last1(f, n):
    f = os.popen("tail -n {1} {0}".format(f, n))
    return pd.read_csv(f, header=None, names=[0, 1, 'subject', 'attachments', 'email'], parse_dates={'dt': [0, 1]}, date_parser=lambda d, t: pd.datetime.strptime(d + ' ' + t, '%Y/%m/%d %H:%M:%S'))

def read_last2(f, n):
    return pd.read_csv(tail(f, n), header=None, names=[0, 1, 'subject', 'attachments', 'email'], parse_dates={'dt': [0, 1]}, date_parser=lambda d, t: pd.datetime.strptime(d + ' ' + t, '%Y/%m/%d %H:%M:%S'))

def tail(fName, n):
    CHUNK_SIZE = 1024
    f = open(fName, 'r')
    LEN = f.seek(0, os.SEEK_END)
    pos = LEN - CHUNK_SIZE
    lines = 0
    while pos >= 0:
        f.seek(pos, os.SEEK_SET)
        s = f.read(CHUNK_SIZE)
        #print('lines = {0}, read chunk'.format(lines))
        lines += s.count('\n')
        if lines <= n:
            pos -= CHUNK_SIZE
        else:
            i = 0
            extra = lines - n
            for x in range(extra):
                i = s.find('\n', i + 1)
            #print('n = {4}: found lines = {0}, extra = {1}, i = {2}, pos = {3}'.format(lines, extra, i, pos, n))
            f.seek(pos + i, os.SEEK_SET)
            return f
    #print('too few lines, return START')
    f.seek(0, os.SEEK_BEGIN)
    return f

%timeit d1 = read_last1('/home/petrum/doc/emails2.csv', 10)
%timeit d2 = read_last2('/home/petrum/doc/emails2.csv', 10)
#(d1.dt == d2.dt).all()

#for i in range(2000):
#    d1 = read_last1('/home/petrum/doc/emails2.csv', i + 1)
#    d2 = read_last2('/home/petrum/doc/emails2.csv', i + 1)
#    if not (d1.dt == d2.dt).all():
#        print(i)
#        break


100 loops, best of 3: 6.94 ms per loop
1000 loops, best of 3: 1.31 ms per loop
